# Configure External Agent as Issuer

In this notebook we configure the external agent as an issuer of PyDentity Multi-Tenant Course certificates by register a public DID onto the Sovrin StagingNet and writing a Credential Definition for the PyDentity Multi-Tenant Tutorial credential schema which is already on the ledger.


## 1. Initialise a controller for Issuer

In [ ]:
%autoawait
import time
import asyncio
import pprint
import sys
from termcolor import colored,cprint
from aiohttp import ClientConnectorError, ClientResponseError
from asyncio import CancelledError

from aries_basic_controller.aries_controller import AriesAgentController
   
# Create a small utility to print json formatted outout more human-readable    
pp = pprint.PrettyPrinter(indent=4)

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8052
ADMIN_URL = "http://external-agent:8051"


In [ ]:
# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)
    

In [ ]:
agent_controller.webhook_listener(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT, webhook_base=WEBHOOK_BASE)

## 2. Generate DID and Write to Sovrin StagingNet

In [ ]:
# generate new DID
response = await agent_controller.wallet.create_did()

try:
    did_object = response['result']
    print("New DID", did_object)
except:
    print("Unexpected error:", sys.exc_info()[0])
    # If you wish to get the error stack trace uncomment the line below and run it again
    # raise

In [ ]:
# write new DID to Sovrin Stagingnet
import requests
import json 

url = 'https://selfserve.sovrin.org/nym'

payload = {"network":"stagingnet","did": did_object["did"],"verkey":did_object["verkey"],"paymentaddr":""}

# Adding empty header as parameters are being sent in payload
headers = {}

try:
    r = requests.post(url, data=json.dumps(payload), headers=headers)
    print(r.json())
    print(r.status_code)
except:
    print("Unexpected error:", sys.exc_info()[0])


## 3. Accepting the Transaction Author Agreement (TAA)

Although the Sovrin StagingNet is permissionless, before DIDs have the authority to write to the ledger they must accept something called a transaction author agreement by signing it using the DID they have on the ledger.

As a global public ledger, the Sovrin Ledger and all its participants are subject to privacy and data protection regulations such as the EU General Data Protection Regulation (GDPR). These regulations require that the participants be explicit about responsibilities for Personal Data.

To clarify these responsibilities and provide protection for all parties, the Sovrin Governance Framework Working Group developed an agreement between Transaction Authors and the Sovrin Foundation. The TAA can be found at Sovrin.org. It ensures that users are aware of and consent to the fact that all data written to the Sovrin Ledger cannot be removed, even if the original author of the transaction requests its removal.

The TAA outlines the policies that users must follow when interacting with the Sovrin Ledger. When a user’s client software is preparing a transaction for submission to the network, it must include a demonstration that the user had the opportunity to review the current TAA and accept it. This is done by including some additional fields in the ledger write transaction: 

* A hash of the agreement
* A date when the agreement was accepted, and
* A string indicating the user interaction that was followed to obtain the acceptance.

The Indy client API used by Sovrin has been extended to allow users to review current and past agreements and to indicate acceptance through an approved user interaction pattern. - source: https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/

For more details on TAA please read more at the following links:
* [Preparing for the Sovrin Transaction Author Agreement](https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/)
* [How the recent approval of the Sovrin Governance Framework v2 affects Transaction Authors
](https://sovrin.org/how-the-recent-approval-of-the-sovrin-governance-framework-v2-affects-transaction-authors/)
* [TAA v2](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/TAA.md)
* [TAA Acceptance Mechanism List (AML)](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/AML.md)

### Fetch The TAA

In [ ]:
try:
    response = await agent_controller.ledger.get_taa()
    TAA = response['result']['taa_record']
    TAA['mechanism'] = "service_agreement"
    print(TAA)
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

### Accept the TAA

In [ ]:
try:
    response = await agent_controller.ledger.accept_taa(TAA)
    ## Will return {} if successful
    print(response)
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

## 4. Set Public DID

Now you have accepted the TAA you should be able to set your DID as public

In [ ]:
try:
    response = await agent_controller.wallet.assign_public_did(did_object["did"])
    pp.pprint(response)
except:
    print("Unexpected error:", sys.exc_info()[0])
#     raise

## 5. Write Credential Definition to Ledger

This is creating the key pair required for the External agent to issue PyDentity Multi-Tenant Tutorial certificates. See schema on the ledger through [IndyScan](https://indyscan.io/tx/SOVRIN_STAGINGNET/domain/195790)

In [ ]:
schema_id = '7DfSpFJMUThna1uQoR1mtY:2:PyDentity Multi-Tenant Tutorial:0.0.1'

In [ ]:
try:
    response = await agent_controller.definitions.write_cred_def(schema_id)
    cred_def_id = response["credential_definition_id"]
    print(cred_def_id)
except ClientConnectorError as err:
    print(err)
except ClientResponseError as err:
    print(err)

**Note: You should be able to see both schema and definition transactions on Sovrin Stagingnet network with [Indyscan](https://indyscan.io/home/SOVRIN_STAGINGNET)**

## 6. Populate the Attributes to Issue to Idenity Holder (User)

We will issue a credential to the identity holder consisting of the following attributes:

credential_attributes = [
    {"name": "full", "value": name},
    {"name": "skill", "value": "PyDentity SSI Ninja"},
    {"name": "age", "value": age}
]

The notebook will ask you to input the identity holder's full name and age which will be used to populate the schema above with the identity holders attribute information.

In [ ]:
name=input("Please enter your name and surname: ")
credential_attributes = [
    {"name": "name", "value": name},
    {"name": "skill", "value": "ACA-Py Multi-Tennancy"},
]
print(credential_attributes)

## 6. Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}
agent_controller.register_listeners([cred_listener, connection_listener], defaults=True)

## 7. Create an Invitation

In [ ]:
# Create Invitation
try:
    invite = await agent_controller.connections.create_invitation(auto_accept=False)
    connection_id = invite["connection_id"]
    print("Connection ID", connection_id)
    print("Invitation Message - Copy This \n\n")
    invite_message = invite['invitation']
    print(invite_message)
    print("\n\n")
except ClientConnectorError as err:
    print(err)
    # raise
except ClientResponseError as err:
    print(err)

### Head over to [Alice](http://localhost:8888/lab/tree/Alice/Part%203%20-%20Communicating%20with%20an%20external%20agent.ipynb) again to accept the invitation

## 7c. Check if established connection is in active state

In [ ]:
import time

try:
    # print('Current state for ConnectionId {} is {}'.format(connection_id,STATE))
    print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
    while STATE != 'active':
    #     print('ConnectionId {0} is not in active state yet'.format(connection_id))
        print(colored("ConnectionId {0} is not in active state yet".format(connection_id), "yellow", attrs=["bold"]))
        trust_ping = await agent_controller.messaging.trust_ping(connection_id,'hello!')
    #     print('Trust ping send to ConnectionId {0} to activate connection'.format(trust_ping))
        print(colored("Trust ping send to ConnectionId {0} to activate connection".format(trust_ping), "blue", attrs=["bold"]))
        time.sleep(5)

    # print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
    print(colored("ConnectionId: {0} is now active. Continue with notebook".format(connection_id), "green", attrs=["bold"]))
except ClientResponseError as err:
    print(err)

## 8. Send Credential


In [ ]:
try:
    record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False)
    record_id = record['credential_exchange_id']
    state = record['state']
    role = record['role']
    credential_ex_id = record['credential_exchange_id']
    credential_id = record['credential_definition_id']
    print("Credential exchange ID: " + credential_ex_id + "\n")
    print("Credential ID: " + credential_id + "\n")
    print()
    print(f"Credential exchange {record_id}, role: {role}, state: {state}")
except CancelledError as err:
    print("Asyncio CancelledError")
except:
    raise

## 9. Accept credential in [Alice's subwallet](http://localhost:8888/lab/tree/Alice/Part%203%20-%20Communicating%20with%20an%20external%20agent.ipynb)

## 10. End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [ ]:
response = await agent_controller.terminate()
print(response)